<a href="https://colab.research.google.com/github/dhyaneesh/Sentinel-1-Man-Made-Change-Detection/blob/main/Change_Detection_and_Edge_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='norse-fragment-435517-g1')

In [11]:
def run_change_detection(roi_geojson, start_date, end_date, significance_level=0.01):
    coords = roi_geojson['features'][0]['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coords)

    im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd'))))

    def clip_img(img):
        return ee.Image(img).clip(aoi)

    im_list = im_coll.toList(im_coll.size()).map(clip_img)

    def selectvv(current):
        return ee.Image(current).select('VV')

    vv_list = im_list.map(selectvv)

    Map = geemap.Map()
    Map.centerObject(aoi, 10)

    rgb_images = (ee.Image.rgb(vv_list.get(0), vv_list.get(1), vv_list.get(2))
                  .log10().multiply(10))
    Map.addLayer(rgb_images, {'min': -20, 'max': 0}, 'RGB Composite')

    def chi2cdf(chi2, df):
        return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

    def omnibus(im_list, m=4.4):
        im_list = ee.List(im_list)
        k = im_list.length()
        klogk = k.multiply(k.log())
        klogk = ee.Image.constant(klogk)
        sumlogs = ee.ImageCollection(im_list.map(lambda img: ee.Image(img).log())).reduce(ee.Reducer.sum())
        logsum = ee.ImageCollection(im_list).reduce(ee.Reducer.sum()).log()
        return klogk.add(sumlogs).subtract(logsum.multiply(k)).multiply(-2*m)

    k = len(im_list.getInfo())
    p_value = ee.Image.constant(1).subtract(chi2cdf(omnibus(vv_list), k - 1))

    change_map = p_value.lt(significance_level)

    Map.addLayer(change_map.updateMask(change_map),
                 {'min': 0, 'max': 1, 'palette': ['black', 'red']},
                 'Change Map')

    object_id = change_map.connectedComponents(connectedness=ee.Kernel.plus(4), maxSize=128)
    Map.add_layer(object_id.randomVisualizer(), None, 'Objects')
    return Map

# Example usage:
roi_geojson ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              80.47171730661205,
              16.50255012168239
            ],
            [
              80.47171730661205,
              16.436584749524357
            ],
            [
              80.5565391560516,
              16.436584749524357
            ],
            [
              80.5565391560516,
              16.50255012168239
            ],
            [
              80.47171730661205,
              16.50255012168239
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
start_date = '2022-01-01'
end_date = '2024-12-31'
significance_level = 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001

result_map = run_change_detection(roi_geojson, start_date, end_date, significance_level)
display(result_map)

Map(center=[16.469569832021776, 80.5141282313297], controls=(WidgetControl(options=['position', 'transparent_b…